In [1]:
import pathlib
import sklearn
import numpy
import nltk
import pandas as pd
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import spacy
nlp = spacy.load('en_core_web_sm')
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [18]:
train_df = pd.read_csv('final-data-extended.csv', delimiter=';')
train_df.head()

,ID,text,sentiment,topic
0,1,A tasty masala .,positive,movie
1,2,unnoticed,negative,movie
2,3,exists only,neutral,movie
3,4,unstinting,negative,movie
4,5,of the funniest jokes of any movie,positive,movie


In [3]:
test_df = pd.read_csv('sentiment-topic-final-test.tsv', delimiter='\t')
test_df.head()

,sentence id,text,sentiment,topic
0,0,It took eight years for Warner Brothers to rec...,negative,movie
1,1,All the New York University students love this...,positive,restaurant
2,2,This Italian place is really trendy but they h...,negative,restaurant
3,3,"In conclusion, my review of this book would be...",positive,book
4,4,The story of this movie is focused on Carl Bra...,neutral,movie


In [4]:
sentences_train = []
sentences_test = []
gold_labels_train = []
gold_labels_test = []

for sentence in train_df['text']:
    sentences_train.append(sentence)

for sentiment in train_df['sentiment']:
    gold_labels_train.append(sentiment)
    
for sentence in test_df['text']:
    sentences_test.append(sentence)

for sentiment in test_df['sentiment']:
    gold_labels_test.append(sentiment)   


In [15]:
def main(train_data, train_gold, test_data, test_gold):
    
    # loading all files as training data.
    text_vec = CountVectorizer(min_df=3, # If a token appears fewer times than this, across all documents, it will be ignored
                                 tokenizer=nltk.word_tokenize)
    
    # Bag of Words representation
    text_counts_train = text_vec.fit_transform(train_data)
    text_counts_test = text_vec.transform(test_data)
    

    # Convert raw frequency counts into TF-IDF values
    tfidf_transformer = TfidfTransformer()
    text_tfidf_train = tfidf_transformer.fit_transform(text_counts_train)
    text_tfidf_test = tfidf_transformer.fit_transform(text_counts_test)
        
    # Train a Multimoda Naive Bayes classifier
    clf = MultinomialNB().fit(text_tfidf_train, train_gold)

    # Predicting the Test set results, find macro recall
    y_pred = clf.predict(text_tfidf_test)

    report = classification_report(test_gold,y_pred,digits = 3)    
    print(report)
    print(test_gold, y_pred)

In [16]:
main(sentences_train, gold_labels_train, sentences_test, gold_labels_test)

C:\Users\Elianne\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

    negative      0.500     0.333     0.400         3
     neutral      0.600     1.000     0.750         3
    positive      1.000     0.750     0.857         4

    accuracy                          0.700        10
   macro avg      0.700     0.694     0.669        10
weighted avg      0.730     0.700     0.688        10

['negative', 'positive', 'negative', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'negative'] ['negative' 'positive' 'neutral' 'negative' 'neutral' 'neutral' 'positive'
 'positive' 'neutral' 'neutral']
